In [1]:
!pip install botocore
!pip install boto3
!pip install tensorflow

In [2]:
import boto3
import botocore
import gzip
import os
import re
import requests
import shutil
import time

import pprint

from botocore.config import Config
from botocore.exceptions import ClientError
from datetime import timezone
from datetime import datetime
from pathlib import Path
from urllib.parse import urlparse

#import tfrecordFormat.save_mrms_as_tfrecord
import xarray as xr
import tensorflow as tf


# This was taken from https://stackoverflow.com/a/62743933
def download(url: str, file_path='', attempts=2):
    """Downloads a URL content into a file (with large file support by streaming)

    :param url: URL to download
    :param file_path: Local file name to contain the data downloaded
    :param attempts: Number of attempts
    :return: New file path. Empty string if the download failed
    """
    if not file_path:
        file_path = os.path.realpath(os.path.basename(url))
    url_sections = urlparse(url)
    if not url_sections.scheme:
        url = f'http://{url}'
    for attempt in range(attempts):
        try:
            with requests.get(url, stream=True) as response:
                response.raise_for_status()
                with open(file_path, 'wb') as out_file:
                    for chunk in response.iter_content(chunk_size=1024*1024): # 1MB chunks
                        out_file.write(chunk)
                return file_path
        except Exception as ex:
            print(f'Attempt #{attempt} failed with error: {ex}')
    return ''

# This was taken from https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html
def s3_upload_file(s3_client, file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        print(f"Upload filed with error: {e}")
        return False
    return True

def get_latest_mrms_datetime():
    url = "https://mrms.ncep.noaa.gov/data/2D/RadarOnly_QPE_01H/"
    response = requests.get(url)

    latest_line = ""
    for line in response.iter_lines():
        if "latest" in str(line):
            latest_line = str(line)

    dt = re.search("\d{2}-\w{3}-\d{4} \d{2}:\d{2}", latest_line).group(0)

    dt_obj = datetime.strptime(dt, "%d-%b-%Y %H:%M")

    return dt_obj

def get_latest_mrms():
    dt_obj = get_latest_mrms_datetime()

    url = "https://mrms.ncep.noaa.gov/data/2D/RadarOnly_QPE_01H/MRMS_RadarOnly_QPE_01H.latest.grib2.gz"

    print('got dt')
    print(dt_obj)
    filename = f"{dt_obj.year}-{dt_obj.month:02}-{dt_obj.day:02}-{dt_obj.hour:02}{dt_obj.minute:02}-radar"

    print(f"Downloading {filename}")
    download(url, f"./{tmp_path}/{filename}.grib2.gz")

    return f"{filename}"

goes_bands = ["07", "08", "09", "10", "13", "14", "16"]
latest_goes = {}
tmp_path = "/home/es_bot/lumo-app/DataFetcher/tmp"

def get_latest_goes(lumo_client, goes_client):
    utc_now = datetime.now(timezone.utc)

    data = goes_client.list_objects_v2(Bucket="noaa-goes16", Prefix=f"ABI-L2-CMIPF/{utc_now.year}/{utc_now.strftime('%j')}/{utc_now.strftime('%H')}")

    if "Contents" not in data:
        return

    contents = data["Contents"]
    contents.sort(key=lambda x: x["LastModified"], reverse=True)

    for band in goes_bands:
        entry = next(x for x in contents if f"M6C{band}" in x["Key"])

        file_time = entry["Key"].split("_")[-1][1:-4]

        dt_obj = datetime.strptime(file_time, "%Y%j%H%M%S")

        filename = f"{dt_obj.year}-{dt_obj.month:02}-{dt_obj.day:02}-{dt_obj.hour:02}{dt_obj.minute:02}-goes-C{band}"

        if filename not in latest_goes[band]:
            print(f"Downloading {filename}")
            goes_client.download_file("noaa-goes16", entry["Key"], f"{tmp_path}/{filename}.nc")

            print(f"Uploading to goes/C{band}/{filename}.nc")
            s3_upload_file(lumo_client, f"{tmp_path}/{filename}.nc", "lumo-app", f"goes/C{band}/{filename}.nc")

            os.remove(f"{tmp_path}/{filename}.nc")

            latest_goes[band].append(filename)

def clean_mrms(lumo_client):
    mrms_files = lumo_client.list_objects(Bucket="lumo-app", Prefix="mrms/")

    if "Contents" not in mrms_files:
        return

    mrms_files = mrms_files["Contents"]
    mrms_files.sort(key=lambda x: x["LastModified"], reverse=True)
    old_files = mrms_files[50:]
    for f in old_files:
        print(f"Deleting {f['Key']}")
        lumo_client.delete_object(Bucket="lumo-app", Key=f["Key"])

def clean_goes(lumo_client):
    for band in goes_bands:
        goes_files = lumo_client.list_objects(Bucket="lumo-app", Prefix=f"goes/C{band}/")

        if "Contents" not in goes_files:
            return

        goes_files = goes_files["Contents"]
        goes_files.sort(key=lambda x: x["LastModified"], reverse=True)
        old_files = goes_files[50:]
        for f in old_files:
            print(f"Deleting {f['Key']}")
            lumo_client.delete_object(Bucket="lumo-app", Key=f["Key"])

    return

2022-03-03 03:58:32.600518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-03 03:58:32.600580: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from tfrecordFormat import test,save_mrms_as_tfrecord

test()

does this work


In [4]:
    path = os.path.join("home", "es_bot", "lumo-app", "DataFetcher", "tmp")
    os.makedirs(path, exist_ok=True)

    #for band in goes_bands:
    #    latest_goes[band] = []

    latest_mrms = []

    #lumo_session = boto3.Session()
    #lumo_client = lumo_session.client("s3")

    #goes_session = boto3.Session()
    #goes_client = goes_session.client('s3', config=Config(signature_version=botocore.UNSIGNED))

    while True:
        mrms_dt = get_latest_mrms_datetime()
        if mrms_dt not in latest_mrms:
            filename = get_latest_mrms()
            with gzip.open(f"./{tmp_path}/{filename}.grib2.gz", 'rb') as f_in:
                with open(f"./{tmp_path}/{filename}.grib2", 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

            ds = xr.open_dataset(f"./{tmp_path}/{filename}.grib2",  engine="cfgrib" )
            print(ds.unknown)
            print(ds.unknown.attrs)
            save_mrms_as_tfrecord( f"./{tmp_path}/{filename}", ds.unknown)

            # Upload to s3
            #print(f"Uploading {filename}")
            #res = s3_upload_file(lumo_client, f"{tmp_path}/{filename}.tfrecord.gz", "lumo-app", f"mrms/{filename}.tfrecord.gz")

            #res = s3_upload_file(lumo_client, f"{tmp_path}/{filename}.grib2", "lumo-app", f"mrms/{filename}.grib2")

            # Delete file
            #os.remove(f"{tmp_path}/{filename}.tfrecord.gz") # are we deleting the grib

            os.remove(f"{tmp_path}/{filename}.grib2") # are we deleting the grib
            os.remove(f"{tmp_path}/{filename}.grib2.gz")

            #latest_mrms.append(mrms_dt)

        #get_latest_goes(lumo_client, goes_client) # un comment 
        #clean_mrms(lumo_client)
        #clean_goes(lumo_client)
        break
        #time.sleep(60 * 2)

got dt
2022-03-03 03:56:00
<xarray.DataArray 'unknown' (latitude: 3500, longitude: 7000)>
[24500000 values with dtype=float32]
Coordinates:
    time            datetime64[ns] ...
    step            timedelta64[ns] ...
    heightAboveSea  float64 ...
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
    valid_time      datetime64[ns] ...
Attributes:
    GRIB_paramId:                             0
    GRIB_dataType:                            ra
    GRIB_numberOfPoints:                      24500000
    GRIB_typeOfLevel:                         heightAboveSea
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  7000
    GRIB_Ny:                                  3500
    GRIB_cfName: 

2022-03-03 04:00:37.901809: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-03 04:00:38.225451: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-03 04:00:38.225523: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d0e9e0cd813c): /proc/driver/nvidia/version does not exist
2022-03-03 04:00:38.228587: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-03 04:00:38.234254: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98000000 excee

In [5]:
ds

<xarray.Dataset>
Dimensions:         (latitude: 3500, longitude: 7000)
Coordinates:
    time            datetime64[ns] ...
    step            timedelta64[ns] ...
    heightAboveSea  float64 ...
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
    valid_time      datetime64[ns] ...
Data variables:
    unknown         (latitude, longitude) float32 -3.0 -3.0 -3.0 ... -3.0 -3.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161
    history:                 2022-03-03T04:00 GRIB to CDM+CF via cfgrib-0.9.1...